In [ ]:
artifact_id_dict = {
    "flowbot": 'run-pjnc1g2v-simulation_metric_table:v827',  # run-{}-train_metric_table:v0
    # "traj1": 'wcx2tyyn',
    "traj5": 'run-zes1lcl9-simulation_metric_table:v829',
    "traj10": 'run-bhto6if3-simulation_metric_table:v820',
    "traj15": 'run-ujqlvh40-simulation_metric_table:v834',
    "traj20": 'run-sk9xeuyz-simulation_metric_table:v835',
}

In [ ]:
import wandb

api = wandb.Api()
for model in artifact_id_dict.keys():
    artifact_id = artifact_id_dict[model]
    artifact = api.artifact(f'r-pad/open_anything_diffusion/{artifact_id}')
    artifact.download()

In [ ]:
import json
import pandas as pd
neat_dfs = pd.DataFrame()
chao_df = pd.DataFrame()
model_order = artifact_id_dict.keys()

desired_columns_order = ['model', 'Table', 'Window', 'Microwave', 'Phone', 'WashingMachine', 'Kettle', 'Toilet', 'Toy', 'Dishwasher', 'Refrigerator', 'Safe', 'Bucket', 'Door', 'TrashCan', 'Laptop', 'Oven', 'KitchenPot', 'FoldingChair', 'Box', 'StorageFurniture', 'Stapler', 'Switch']

for model in artifact_id_dict.keys():
    artifact_id = artifact_id_dict[model]
    artifact_local_path = f'/home/yishu/open_anything_diffusion/notebooks/artifacts/{artifact_id}/simulation_metric_table.table.json'
    with open(artifact_local_path) as file:
        json_dict = json.load(file)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"]).T
    df.columns = df.loc['index']
    print(list(df.columns))
    df.drop('index', inplace=True)
    df['model'] = model
    df = df.reindex(columns=desired_columns_order)
    chao_df = pd.concat([chao_df, df])
for metric in ["count", "success_rate", "norm_dist"]:
    new_df = chao_df.loc[metric].sort_values(by='model', key=lambda x: x.map({v: i for i, v in enumerate(model_order)}))
    neat_dfs = pd.concat([neat_dfs, new_df])

In [ ]:
neat_dfs.to_excel('./evaluation_matrix.xlsx')